---
### Step 0: Load The Data

In [1]:
# Import libraries
import math
import cv2
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Lambda, Conv2D, Activation, Dropout
from keras.callbacks import ModelCheckpoint
%matplotlib inline

Using TensorFlow backend.


In [2]:
#img_data_set = []
#label_data_set = []
lines = []
with open('labeled_images.csv') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for line in reader:
            lines.append(line)
print(len(lines))

4110


In [3]:
#labeled_pictures = pd.read_csv('labeled_images.csv')
shuffle(lines)
train_data, val_data = train_test_split(lines, test_size = 0.2)
print(len(train_data))
print(len(val_data))

3288
822


---
### Step 1: Define Data Generator

In [4]:
def data_generator (data_lines, batch_size = 40) :
    num_lines = len(data_lines)
    while True:
        shuffle(data_lines)
        for offset in range(0, num_lines, batch_size):
            batch_lines = data_lines[offset:offset + batch_size]
            images = []
            labels = []
            for batch_line in batch_lines:
                img = cv2.cvtColor(cv2.imread('./rec_images/'+str(batch_line[0])+'.jpeg'), cv2.COLOR_BGR2HSV)
                images.append(img)
                labels.append(batch_line[1])
            X = np.array(images)
            y = to_categorical(np.array(labels), num_classes=5)
            yield shuffle(X, y)

---
### Step 2: Define Model

In [10]:
model = Sequential()
#normalize data
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(600,800,3)))

model.add(Conv2D(24,(5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(36,(5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(48,(5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam')
model.fit_generator(generator = data_generator(train_data),
                    validation_data = data_generator(val_data),
                    epochs = 10,
                    steps_per_epoch  = math.ceil(len(train_data) / 128),
                    validation_steps = math.ceil(len(val_data)   / 128)    )
model.save('model.h5')
print('Model saved')

Epoch 1/10
26/26 [==============================] - 17s 635ms/step - loss: 1.3061 - val_loss: 0.8058
Epoch 2/10
26/26 [==============================] - 16s 605ms/step - loss: 0.6093 - val_loss: 0.2359
Epoch 3/10
26/26 [==============================] - 16s 599ms/step - loss: 0.2378 - val_loss: 0.2344
Epoch 4/10
26/26 [==============================] - 15s 595ms/step - loss: 0.1020 - val_loss: 0.0357
Epoch 5/10
26/26 [==============================] - 16s 609ms/step - loss: 0.0707 - val_loss: 0.0102
Epoch 6/10
26/26 [==============================] - 16s 601ms/step - loss: 0.0579 - val_loss: 0.0192
Epoch 7/10
26/26 [==============================] - 15s 592ms/step - loss: 0.0208 - val_loss: 0.0019
Epoch 8/10
26/26 [==============================] - 16s 604ms/step - loss: 0.0246 - val_loss: 0.0064
Epoch 9/10
26/26 [==============================] - 16s 600ms/step - loss: 0.0505 - val_loss: 0.0022
Epoch 10/10
26/26 [==============================] - 15s 589ms/step - loss: 0.0197 - val_lo

In [12]:
test_images = np.array([cv2.cvtColor(cv2.imread('./rec_images/5822.jpeg'), cv2.COLOR_BGR2HSV)])
pred_vects = model.predict(test_images)
print("Predicted Probabilities: {}".format(pred_vects))
pred_labels = pred_vects.argmax(axis=-1)
print("Predicted Labels: {}".format(pred_labels))

Predicted Probabilities: [[1.6459733e-08 6.5546340e-07 9.9999928e-01 6.5244851e-11 7.5452611e-10]]
Predicted Labels: [2]
